In [55]:
import sys

import pandas as pd
import numpy as np

from bertopic import BERTopic

import asyncio

from text_topic_extraction.concept_extractor.concept_extractor import Concept, PatientConcepts
from text_topic_extraction.text_topic_processor import TextTopicProcessor, AsyncTextTopicProcessor, BERTopicModeler, QuestionGenerator, AnswerGenerator
from text_topic_extraction.config import API_VERSION, AZURE_ENDPOINT, OPENAI_API_KEY, USE_AZURE

In [2]:
np.random.seed(0)

In [3]:
question_prompt = """
The following topics were found in clinical notes. Generate yes/no questions for the following medical topics:
{topics_text}
Return a JSON object with topic numbers as keys and questions as values. Example:
{
"1": "Does the note mention the patient having a history of diabetes?"
}
"""

answer_prompt = """
You will be given a clinical note. I will give you a series of questions. Your task is answer each question with a probability from 0 to 1. Summarize the response with a JSON that includes your answer to all of the questions. Questions:
{prompt_questions}

clinical note:
{sentence}

Example answer: {
    "0": 0,
    "1": 1,
    "2": 0.5
}
Answer all the questions and do not answer with anything else besides valid JSON. Do not add comments to the JSON.
"""

In [4]:
llm_extract_df = pd.read_csv("../llm-bart/exp_mimic/_output/long_notes/max_obs_-1/seed_0/gpt-4o-mini/concept_extractions.csv")

In [67]:
llm_extract_df.llm_output

0       male, man, gentleman,age 82, elderly, senior,s...
1       81-year-old, elderly, senior,female, woman, ad...
2       75 years, elderly, senior,man, male,retired, u...
3       66 year old, elderly, senior,male, man, adult,...
4       female, woman, patient,elderly, senior, aged,m...
                              ...                        
7038    tobacco use, smoking, nicotine use,15 pack yea...
7039    86 M, elderly, senior,prostate cancer, cancer,...
7040    86 M, elderly, senior,prostate cancer, cancer,...
7041    elderly, senior, advanced age,female, woman,fe...
7042    male, man, gender,68 years old, elderly, senio...
Name: llm_output, Length: 7043, dtype: object

In [5]:
rand_idxs = np.random.choice(llm_extract_df.shape[0], llm_extract_df.shape[0], replace=False)
train_idxs = rand_idxs[:400]
test_idxs = rand_idxs[400:800]

In [31]:
patient_notes = {}
for patient_id in train_idxs:
    note_text = llm_extract_df.iloc[patient_id].sentence
    note_type = "H&P"
    patient_notes[str(patient_id)] = [{"note_text": note_text, "note_type": note_type}]

In [ ]:
concept_id = 0
patient_concepts = {}
for patient_id in train_idxs:
    a = llm_extract_df.iloc[patient_id].llm_output.split(",")
    concept_list = [Concept(id=str(concept_id + concept_offset_id), descriptor=protoconcept.strip().lower()) for concept_offset_id, protoconcept in enumerate(a)]
    concept_id += len(concept_list)
    patient_concepts[str(patient_id)] = PatientConcepts(
        patient_id=str(patient_id),
        all_concepts=concept_list
    )

{'2200': PatientConcepts(patient_id='2200', all_concepts=[Concept(id='0', descriptor='nkda', synonyms=[], source='llm', patient_id='', note_id='', metadata={}), Concept(id='1', descriptor='no known drug allergies', synonyms=[], source='llm', patient_id='', note_id='', metadata={}), Concept(id='2', descriptor='social history', synonyms=[], source='llm', patient_id='', note_id='', metadata={}), Concept(id='3', descriptor='lifestyle factors', synonyms=[], source='llm', patient_id='', note_id='', metadata={}), Concept(id='4', descriptor='history', synonyms=[], source='llm', patient_id='', note_id='', metadata={}), Concept(id='5', descriptor='medical history', synonyms=[], source='llm', patient_id='', note_id='', metadata={}), Concept(id='6', descriptor='demographics', synonyms=[], source='llm', patient_id='', note_id='', metadata={}), Concept(id='7', descriptor='population data', synonyms=[], source='llm', patient_id='', note_id='', metadata={}), Concept(id='8', descriptor='health status',

In [58]:
processor = TextTopicProcessor(
    model_type="gpt-4o-mini-2024-07-18", #"gpt-4o-2024-08-06",
    temperature=0.0,
    num_topics=None,
    use_cache=True,
    use_structured_output=True,
    use_azure=False,
    azure_endpoint=AZURE_ENDPOINT,
    api_version=API_VERSION,
    api_key=OPENAI_API_KEY,
    cache_dir="cache",
    concept_prompt_template="{notes}",
    concept_questions_prompt_template=question_prompt,
    answer_prompt_template=answer_prompt
)

2025-03-13 07:47:41,575 - text_topic_extraction.cache.cache_manager - INFO - Text Topic Extraction cache initialized at cache/text_topic_cache.duckdb


2025-03-13 07:47:41,585 - text_topic_extraction.text_topic_processor - INFO - Initialized TextTopicProcessor with model gpt-4o-mini-2024-07-18


In [59]:
processor.patient_notes = patient_notes

In [60]:
processor.openai_client.client

In [61]:
topic_modeler = BERTopicModeler(
    num_topics=processor.num_topics,
    embedding_model=(
        processor.embedding_model if processor.embedding_model != "default" else None
    ),
    vectorizer_model=processor.vectorizer_model,
    min_topic_size=processor.min_topic_size,
    language=processor.language,
    n_gram_range=processor.n_gram_range,
)

question_generator = QuestionGenerator(
    llm_api=processor.openai_client,
    batch_prompt_template=processor.concept_questions_prompt_template,
)
answer_generator = AnswerGenerator(
    llm_api=processor.openai_client, prompt_template=processor.answer_prompt_template
)

2025-03-13 07:47:54,119 - text_topic_extraction.topic_modeler.bertopic_modeler - INFO - Initialized BERTopicModeler with None topics


In [9]:
topics = processor._perform_topic_modeling(
    patient_concepts=patient_concepts,
    topic_modeler=topic_modeler,
)

2025-03-13 06:15:30,364 - text_topic_extraction.text_topic_processor - INFO - Performing topic modeling on concepts from 400 patients
2025-03-13 06:15:30,393 - sentence_transformers.SentenceTransformer - INFO - Use pytorch device_name: mps
2025-03-13 06:15:30,393 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:

In [20]:
batch_size = 100
all_topic_questions = []
for i in range(0, len(topics), batch_size):
    topic_batch = topics[i:i + batch_size]
    topic_questions = processor._generate_questions(
        topics=topic_batch,
        question_generator=question_generator,
    )
    print("topic question ex:", topic_questions[0])
    print("num topics", len(topic_questions))
    all_topic_questions += topic_questions

2025-03-13 07:06:41,788 - text_topic_extraction.text_topic_processor - INFO - Generating questions for 100 topics
2025-03-13 07:07:25,019 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-13 07:07:25,073 - text_topic_extraction.cache.cache_manager - INFO - Stored response in cache with key 2db52454ab...
2025-03-13 07:07:25,075 - text_topic_extraction.text_topic_processor - INFO - Generated 100 questions
2025-03-13 07:07:25,075 - text_topic_extraction.text_topic_processor - INFO - Question for topic 0: Is the patient described as an elderly or aging person?
2025-03-13 07:07:25,075 - text_topic_extraction.text_topic_processor - INFO - Question for topic 1: Is the patient identified as a male cyclist?
2025-03-13 07:07:25,075 - text_topic_extraction.text_topic_processor - INFO - Question for topic 2: Is the patient's social history unknown?
2025-03-13 07:07:25,075 - text_topic_extraction.text_topic_processor - INFO - Question for topic

topic question ex: topic_id=0 topic_name='elderly aging person' question='Is the patient described as an elderly or aging person?' keywords=['elderly', 'aging', 'person', 'population', 'worker', 'mother', 'aged', '', '', ''] keyword_scores=[0.13809806533186764, 0.010444242025935841, 0.009521822757240776, 0.008182954483843301, 0.007694280806424764, 0.007232136342474878, 0.005685869822041315, 1e-05, 1e-05, 1e-05]
num topics 100


2025-03-13 07:07:58,180 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-13 07:07:58,214 - text_topic_extraction.cache.cache_manager - INFO - Stored response in cache with key ea74eadeb4...
2025-03-13 07:07:58,215 - text_topic_extraction.text_topic_processor - INFO - Generated 100 questions
2025-03-13 07:07:58,215 - text_topic_extraction.text_topic_processor - INFO - Question for topic 100: Is the patient retired or semiretired?
2025-03-13 07:07:58,216 - text_topic_extraction.text_topic_processor - INFO - Question for topic 101: Does the patient receive support from welfare or social services?
2025-03-13 07:07:58,216 - text_topic_extraction.text_topic_processor - INFO - Question for topic 102: Does the patient have any neurological conditions such as sclerosis or Parkinson's?
2025-03-13 07:07:58,216 - text_topic_extraction.text_topic_processor - INFO - Question for topic 103: Is the patient currently afebrile?
2025-03-13 07:07:58,

topic question ex: topic_id=100 topic_name='retired semiretired ' question='Is the patient retired or semiretired?' keywords=['retired', 'semiretired', '', '', '', '', '', '', '', ''] keyword_scores=[0.44407066909305615, 0.0979544590171156, 1e-05, 1e-05, 1e-05, 1e-05, 1e-05, 1e-05, 1e-05, 1e-05]
num topics 100


2025-03-13 07:08:31,995 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-13 07:08:32,024 - text_topic_extraction.cache.cache_manager - INFO - Stored response in cache with key 014ba907c1...
2025-03-13 07:08:32,025 - text_topic_extraction.text_topic_processor - INFO - Generated 100 questions
2025-03-13 07:08:32,025 - text_topic_extraction.text_topic_processor - INFO - Question for topic 200: Is there a mention of a blockage or blocked artery?
2025-03-13 07:08:32,026 - text_topic_extraction.text_topic_processor - INFO - Question for topic 201: Is the location of interest near a door or in North Carolina?
2025-03-13 07:08:32,026 - text_topic_extraction.text_topic_processor - INFO - Question for topic 202: Is there a condition affecting the heart or cardiac function?
2025-03-13 07:08:32,027 - text_topic_extraction.text_topic_processor - INFO - Question for topic 203: Does the CXR show any proximal findings?
2025-03-13 07:08:32,027 - t

...

topic question ex: topic_id=1100 topic_name='overload volume buildup' question='Is there evidence of fluid overload or volume buildup in the patient?' keywords=['overload', 'volume', 'buildup', 'control', 'fluid', '', '', '', '', ''] keyword_scores=[0.6536134893969836, 0.34657359027997264, 0.2203975327885101, 0.060909977499883174, 0.055409665033697375, 1e-05, 1e-05, 1e-05, 1e-05, 1e-05]
num topics 100


2025-03-13 07:14:35,763 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-13 07:14:35,811 - text_topic_extraction.cache.cache_manager - INFO - Stored response in cache with key bc5b5583ab...
2025-03-13 07:14:35,812 - text_topic_extraction.text_topic_processor - INFO - Generated 100 questions
2025-03-13 07:14:35,812 - text_topic_extraction.text_topic_processor - INFO - Question for topic 1200: Is the patient a female in her 80s, 60s, or 50s?
2025-03-13 07:14:35,813 - text_topic_extraction.text_topic_processor - INFO - Question for topic 1201: Are there any communication challenges mentioned in the notes?
2025-03-13 07:14:35,813 - text_topic_extraction.text_topic_processor - INFO - Question for topic 1202: Was an abdominal ultrasound performed?
2025-03-13 07:14:35,813 - text_topic_extraction.text_topic_processor - INFO - Question for topic 1203: Is there a mention of deep vein thrombosis?
2025-03-13 07:14:35,814 - text_topic_extracti

topic question ex: topic_id=1200 topic_name='81f 60f 52f' question='Is the patient a female in her 80s, 60s, or 50s?' keywords=['81f', '60f', '52f', '40s50s', '25f', '41', '', '', '', ''] keyword_scores=[0.8177716802655624, 0.5037657892308802, 0.5037657892308802, 0.5037657892308802, 0.5037657892308802, 0.35498666425542863, 1e-05, 1e-05, 1e-05, 1e-05]
num topics 100


2025-03-13 07:14:49,080 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-03-13 07:14:49,108 - text_topic_extraction.cache.cache_manager - INFO - Stored response in cache with key c4edb2d511...
2025-03-13 07:14:49,110 - text_topic_extraction.text_topic_processor - INFO - Generated 19 questions
2025-03-13 07:14:49,110 - text_topic_extraction.text_topic_processor - INFO - Question for topic 1300: Is the patient receiving nonhospital medical treatment?
2025-03-13 07:14:49,110 - text_topic_extraction.text_topic_processor - INFO - Question for topic 1301: Was a tissue sample taken for analysis?
2025-03-13 07:14:49,110 - text_topic_extraction.text_topic_processor - INFO - Question for topic 1302: Is the severity of the condition considered medium or average?
2025-03-13 07:14:49,111 - text_topic_extraction.text_topic_processor - INFO - Question for topic 1303: Is there any mention of obstructed airways or obtundation?
2025-03-13 07:14:49,111

topic question ex: topic_id=1300 topic_name='nonhospital treatment medical' question='Is the patient receiving nonhospital medical treatment?' keywords=['nonhospital', 'treatment', 'medical', '', '', '', '', '', '', ''] keyword_scores=[0.2385167400774557, 0.07497007364545744, 0.03210455534029746, 1e-05, 1e-05, 1e-05, 1e-05, 1e-05, 1e-05, 1e-05]
num topics 19


In [39]:
patient_id = str(train_idxs[0])
topic_answers = answer_generator.generate_answers(
    patient_concepts=patient_concepts[patient_id],
    topic_questions=all_topic_questions,
    patient_notes=patient_notes[patient_id],
)

2025-03-13 07:34:03,469 - text_topic_extraction.answer_generator.answer_generator - INFO - Generating answers for patient 2200 for 1319 questions
2025-03-13 07:34:03,470 - text_topic_extraction.answer_generator.answer_generator - INFO - Sending batch of 1319 questions for patient 2200
2025-03-13 07:34:03,485 - text_topic_extraction.llm_api.openai_client - INFO - Using cached structured response (key: 3ebf4b8fa8...)


In [68]:
topic_answers

[TopicAnswer(topic_id=0, topic_name='elderly aging person', question='Is the patient described as an elderly or aging person?', probability=0.0, answer=False),
 TopicAnswer(topic_id=1, topic_name='male cyclist individual', question='Is the patient identified as a male cyclist?', probability=0.0, answer=False),
 TopicAnswer(topic_id=2, topic_name='social history unknown', question="Is the patient's social history unknown?", probability=1.0, answer=True),
 TopicAnswer(topic_id=3, topic_name='senior gentleman man', question='Is the patient referred to as a senior gentleman or man?', probability=0.0, answer=False),
 TopicAnswer(topic_id=4, topic_name='man lad ', question='Is the patient described as a man or lad?', probability=0.0, answer=False),
 TopicAnswer(topic_id=5, topic_name='female girl 75y', question='Is the patient a 75-year-old female or girl?', probability=0.0, answer=False),
 TopicAnswer(topic_id=6, topic_name='woman lady she', question='Is the patient referred to as a woman o

In [65]:
patient_concepts_subset = {str(idx): patient_concepts[str(idx)] for idx in train_idxs[:50]}
processor.patient_notes = {str(idx): patient_notes[str(idx)] for idx in train_idxs[:50]}

patient_answers = processor._generate_answers(
    patient_concepts=patient_concepts_subset,
    topic_questions=all_topic_questions,
    answer_generator=answer_generator,
)

2025-03-13 07:50:24,451 - text_topic_extraction.text_topic_processor - INFO - Generating answers for 50 patients
2025-03-13 07:50:24,451 - text_topic_extraction.text_topic_processor - INFO - Generating answers for patient 2200
2025-03-13 07:50:24,452 - text_topic_extraction.answer_generator.answer_generator - INFO - Generating answers for patient 2200 for 1319 questions
2025-03-13 07:50:24,453 - text_topic_extraction.answer_generator.answer_generator - INFO - Sending batch of 1319 questions for patient 2200
2025-03-13 07:50:24,465 - text_topic_extraction.llm_api.openai_client - INFO - Using cached structured response (key: 3ebf4b8fa8...)
2025-03-13 07:50:24,470 - text_topic_extraction.text_topic_processor - INFO - Generating answers for patient 4627
2025-03-13 07:50:24,470 - text_topic_extraction.answer_generator.answer_generator - INFO - Generating answers for patient 4627 for 1319 questions
2025-03-13 07:50:24,471 - text_topic_extraction.answer_generator.answer_generator - INFO - Sen

KeyboardInterrupt: 

In [ ]:
# Create topic matrix
processor.topic_matrix = answer_generator.create_topic_matrix(patient_answers)

# Create matrix DataFrames (always do this even if not saving)
processor.binary_matrix_df = MatrixBuilder.create_binary_matrix(topic_matrix)
processor.probability_matrix_df = MatrixBuilder.create_probability_matrix(
    topic_matrix
)

# Create return dictionary with DataFrames
results = {
    "binary_matrix_df": binary_matrix_df,
    "probability_matrix_df": probability_matrix_df,
    "topic_matrix": topic_matrix,
    "topics": topics,
    "topic_questions": topic_questions,
}

''

In [ ]:
processor._save_results(
    patient_concepts=patient_concepts,
    topics=topics,
    topic_questions=topic_questions,
    patient_answers=patient_answers,
    topic_matrix=topic_matrix,
    output_dir="exp_mimic/_output",
    save_intermediate=False,
)